Let's start by reading in the data

In [1]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


#We assume data is in a parallel directory to this one called 'data'
cwd = os.getcwd()
datadir = '/'.join(cwd.split('/')[0:-1]) + '/data/'
#or you can hardcode the directory
#datadir = 

print(datadir)

/Users/ruojunhong/Desktop/DataScienceCourse/ipython/data/


Now read in the data called survey_responses_2016.csv into a pandas data frame.

In [2]:
#Student put in read data command here:
data = pd.read_csv('/Users/ruojunhong/Desktop/DataScienceCourse/ipython/data/survey_responses_2016.csv',header=0,sep=',')
data.head()

,id,cs_python,cs_java,cs_c,cs_perl,cs_javascript,cs_r,cs_sas,profile_1,profile_2,profile_3,profile_4,profile_5,profile_6,profile_7,fruit,len_answer,season,experience_coded,experience
0,1,1,1,0,0,0,1,0,1,5,6,5,1,6,5,Oranges,136,Fall,2,"< 2 years, I'm fresh!"
1,2,0,1,0,0,0,0,0,1,3,6,4,1,5,5,Apples,112,Summer,1,"None, I just finished my undergrad!"
2,3,1,0,1,0,0,1,1,1,4,4,7,1,1,5,Oranges,97,Spring,1,"None, I just finished my undergrad!"
3,4,0,0,1,0,0,1,0,2,1,8,7,2,5,5,Oranges,1879,Winter,1,"None, I just finished my undergrad!"
4,5,1,0,0,0,1,1,0,5,4,6,6,6,1,7,Apples,699,Spring,2,"< 2 years, I'm fresh!"


Let's look at the column headers and use something more descriptive

In [3]:
#Student put in code to look at column names
data.columns

Index(['id', 'cs_python', 'cs_java', 'cs_c', 'cs_perl', 'cs_javascript',
       'cs_r', 'cs_sas', 'profile_1', 'profile_2', 'profile_3', 'profile_4',
       'profile_5', 'profile_6', 'profile_7', 'fruit', 'len_answer', 'season',
       'experience_coded', 'experience'],
      dtype='object')

Column names like 'profile_1-profile_7' aren't very descriptive. As a quick data maintenance task, let's rename the columns starting with 'profile'. The dictionary in the next cell maps the integer index to a descriptive text.

Tactically, let's loop through each column name. Within the loop let's check whether the column name starts with 'profile.' If it does, let's create a new name that swaps the key with the value using profile_mapping dictionary (i.e., profile_1 -> profile_Viz). We then add the new column name to a list. If it doesn't start with 'profile' just add the old column name to the list. 

In [4]:
profile_mapping = {1:'Viz',
                   2:'CS',
                   3:'Math',
                   4:'Stats',
                   5:'ML',
                   6:'Bus',
                   7:'Com'}

In [5]:
#Student put code here to change the header names
newcols = []

for colname in data.columns:
    #finish the loop  
    if re.search("profile",colname):
        newcols.append('profile_()'.format(profile_mapping[int(colname[-1])]))
    else:
        newcols.append(colname)

data.columns = newcols

    
#Now swap the old columns with the values in newcols    


In [6]:
data.columns

Index(['id', 'cs_python', 'cs_java', 'cs_c', 'cs_perl', 'cs_javascript',
       'cs_r', 'cs_sas', 'profile_()', 'profile_()', 'profile_()',
       'profile_()', 'profile_()', 'profile_()', 'profile_()', 'fruit',
       'len_answer', 'season', 'experience_coded', 'experience'],
      dtype='object')

Let's use this data to illustrate common data analytic techniques. We have one numeric variable (len_answer) and different categorical variables which may carry some signal of the 'len_answer' variable. 

'Len_answer' is the character count of the response to the following question: "Besides the examples given in lecture 1, discuss a case where data science has created value for some company. Please explain the company's goals and how any sort of data analysis could have helped the company achieve said goals." As this is a subjective business question, let's hypothesize that students with more professional experience might be more likely to give longer answers. 

In more technical terms, we'll test whether the variance of len_answer can be explained away by the categorical representation of a student's experience. 

The first thing we should do is look at the distribution of len_answer.

In [7]:
#Student - plot a histogram here for len_answer


It looks like we have at least one strong outlier and a thick distribution around 0. Let's also use the Pandas describe() method to get a stronger sense of the distribution.

In [8]:
data.len_answer.describe()

count     102.000000
mean      547.725490
std       480.267152
min         0.000000
25%       262.500000
50%       460.500000
75%       745.750000
max      3683.000000
Name: len_answer, dtype: float64

Let's consider cleaning up the data. We'll remove the max value as well as those with a length less than 20 (which we think is a generous minimum to communicate a reasonable answer.

Create a new data_frame that removes these outliers.

In [14]:
#Student create a filtered data frame here
outlier_filter = (data.len_answer>20)&(data.len_answer<data.len_answer.max())
data_clean = data[outlier_filter]
#
data_clean.shape

(93, 20)

Now that we have cleaned our data, let's run a pairwise t-test on each experience level to see if their difference in len_answer is statistically significant. To run a t-test, we'll need the mean, standard-deviation and count for each group. We can achieve this with a pandas groupby operation.

In [15]:
#Student input code here
data_clean_grouped = data_clean[['len_answer','experience']].groupby(['experience']).agg(['mean','std','count'])
#run this to look at the grouped df
data_clean_grouped

len_answer                  
                                                 mean         std count
experience                                                             
2-5 years, I'm getting good at what I do!  732.222222  398.570468    18
5+ years, I'm a veteran!                   717.333333  269.793748     6
< 2 years, I'm fresh!                      489.312500  285.271501    16
None, I just finished my undergrad!        507.000000  335.536253    53

Visually, we can see a potential split between the [0, 2] year experience range and the [2+] experience range. Let's be more rigorous and run t-tests. Let's write a function that takes in the necessary statistics and returns a p-value.

Remember, the t-stat for the difference between two means is:

<center>$t = \frac{\hat{\mu_1} - \hat{\mu_2}}{\sqrt{\frac{\hat{\sigma_1}^2}{n_1} + \frac{\hat{\sigma_2}^2}{n_2}}}$</center>

The p-value can be found using a t-distribution, but for simplicity, let's approximate this with the normal distribution. For the 2-tailed test, the p-value is: 2 * (1 - Norm.CDF(T)).

In [18]:
#Student complete the function
from scipy.stats import norm
def pvalue_diffmeans_twotail(mu1, sig1, n1, mu2, sig2, n2):
    '''
    P-value calculator for the hypothesis test of mu1 != mu2.
    Takes in the approprate inputs to compute the t-statistic for the difference between means
    Outputs a p-value for a two-sample t-test.
    '''
    t =( mu1-mu2)/np.sqrt(sig1*sig1/n1+sig2*sig2/n2)
    diff = mu1-mu2
    stderr = np.sqrt(sig1**2/n1*sig2**2/n2)
    t = diff/stderr
    
    p_value = 2*(1-norm.cdf(t))
    
    return (t, p_value)

In [19]:
data_clean_grouped.index.values

array(["2-5 years, I'm getting good at what I do!",
       "5+ years, I'm a veteran!", "< 2 years, I'm fresh!",
       'None, I just finished my undergrad!'], dtype=object)

Now loop through all possible pairs in data_clean_grouped and perform a t-test.

In [21]:
#Student put in code here:


#get distinct values in the data frame for the experience variable
grps = data_clean_grouped.index.values

#Now loop through each pair
for i, grp1 in enumerate(grps):
    for grp2 in grps[i + 1:]:
    
        '''
        Also, the result of groupby uses a multi-index. So be sure to index on 'len_answer' as well.
        Then pull out the mean, std, and cnt from that result.   
        '''        

        #some code should go here
        row1 = data_clean_grouped.ix[grp1].ix['lne_answer']
        row2 = data_clean_grouped.ix[grp2].ix['len_answer']
        tstat, p_value = pvalue_diffmeans_twotail(row1['mean'],row1['std'],row1['count'],row2['mean'],row2['std'],row2['count'])
        print('Two tailed T-Test between groups: {} and {}'.format(grp1, grp2))
        print('Diff = {} characters'.format(round(row1['mean'] - row2['mean'], 0)))
        print('The t-stat is {} and p-value is {}'.format(round(tstat, 3), round(p_value, 3)))
        print('')

KeyError: 'lne_answer'

What are some observations you might have about the above results? Are there any with large deviances that are not statistically significant at at least a 95% level?

Also, how do the numbers change if you rerun it using the original data, and not the cleaned data. What is the effect of outliers on the results?

In [11]:
#Rerun everything without cleaning outliers

grps = 

#Now loop through each pair
for i, grp1 in enumerate(grps):
    for grp2 in grps[i + 1:]:
    
        '''
        Also, the result of groupby uses a multi-index. So be sure to index on 'len_answer' as well.
        Then pull out the mean, std, and cnt from that result.   
        '''        
    
        print('Two tailed T-Test between groups: {} and {}'.format(grp1, grp2))
        print('Diff = {} characters'.format(round(row1['mean'] - row2['mean'], 0)))
        print('The t-stat is {} and p-value is {}'.format(round(tstat, 3), round(p_value, 3)))
        print('')